In [1]:
from chronos_utils import *

In [2]:
samples_local_path = 'mit-bih-arrhythmia-database-1.0.0'
observation_names = sorted(list(set([x[:-4] for x in os.listdir(samples_local_path) if (((x[0]=='2') | (x[0] == '1')) & (len(x) == 7))])))

In [3]:
save_dict = {}
for filename_base in observation_names:
    file_path = samples_local_path + '/' + filename_base

    try:
        MLII = create_record(filepath = file_path, column_subset=['MLII'])
        MLII_quant = quantile_outlier_removal(MLII, column_name = 'MLII', quantile_min=0.00001, quantile_max = 0.99999)
        MLII_SMA = simple_moving_average(MLII_quant, 3)
        MLII_medfilt = median_filtering(MLII_SMA['MLII'],3)
        MLII_sos = sos_filter(MLII_medfilt)
        MLII_notch = notch_filtering(MLII_sos)
        final_df = normalizer(MLII_notch)
        save_dict.update({'a' + filename_base : final_df})
    except KeyError:
        print('KeyError for observation', filename_base, '- skipping this observation.')


KeyError for observation 102 - skipping this observation.
KeyError for observation 104 - skipping this observation.


In [4]:
np.savez_compressed('MIT-BIH.npz', **save_dict)

In [6]:
with np.load('MIT-BIH.npz') as data:
    print(data.files)
    a = data['a100']


a

['a100', 'a101', 'a103', 'a105', 'a106', 'a107', 'a108', 'a109', 'a111', 'a112', 'a113', 'a114', 'a115', 'a116', 'a117', 'a118', 'a119', 'a121', 'a122', 'a123', 'a124', 'a200', 'a201', 'a202', 'a203', 'a205', 'a207', 'a208', 'a209', 'a210', 'a212', 'a213', 'a214', 'a215', 'a217', 'a219', 'a220', 'a221', 'a222', 'a223', 'a228', 'a230', 'a231', 'a232', 'a233', 'a234']


array([0.35991953, 0.3479142 , 0.33054982, ..., 0.63243908, 0.49723198,
       0.34611849])